In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# uncomment this to read and process all excel files again

tennis_df = pd.DataFrame()
for year in range(2001, 2022):
    
    file = ""
    if year < 2013:
        file = f"{year}.xls"
        print(file)
    else:
        assert(year >= 2013)
        file = f"{year}.xlsx"
        print(file)
    
    year_df = pd.read_excel(file)
    year_df['Year'] = year
    tennis_df = tennis_df.append(year_df)


2001.xls
2002.xls
2003.xls
2004.xls
2005.xls
2006.xls
2007.xls
2008.xls
2009.xls
2010.xls
2011.xls
2012.xls
2013.xlsx


C:\Users\briel\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


2014.xlsx
2015.xlsx
2016.xlsx
2017.xlsx
2018.xlsx
2019.xlsx
2020.xlsx
2021.xlsx


In [3]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,Year,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.0,1.1,3.5,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.3,1.12,3.8,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# file_name = "tennis_data_2001_2021_raw.xlsx"
# tennis_df.to_excel(file_name) 


In [5]:
# tennis_df = pd.read_excel(file_name)

In [6]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,Year,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.0,1.1,3.5,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.3,1.12,3.8,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# as of right now, don't see these columns as adding much to the model/data
# once we can get this to work with BET365 then we can look at other books
cols_list_drop = ['Best of', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
        'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL',
       'WPts', 'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW',
       'MaxL', 'AvgW', 'AvgL']

In [8]:
tennis_df = tennis_df.drop(cols_list_drop, axis=1)

In [9]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Clement A.,Gaudenzi A.,18.0,101,2001,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Goldstein P.,Jones A.,81.0,442,2001,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Haas T.,Smith L.,23.0,485,2001,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Henman T.,Rusedski G.,10.0,69,2001,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Hewitt L.,Arthurs W.,7.0,83,2001,NaN,NaN


In [10]:
tennis_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'Year', 'B365W', 'B365L'],
      dtype='object')

In [11]:
len(tennis_df)

55604

In [12]:
# for year, frame in tennis_df.groupby('Year'):
#     percent_missing = frame.isnull().sum() * 100 / len(frame)
#     missing_value_df = pd.DataFrame({'column_name': frame.columns,
#                                  'percent_missing': percent_missing})
#     print(year)
#     display(missing_value_df)

In [13]:
# dropping all na's because the only NA's were in B365 and in rank - can't run model without either of these
tennis_df = tennis_df.dropna()

In [14]:
# make sure that the drop na worked
percent_missing = tennis_df.isnull().sum() * 100 / len(tennis_df)
missing_value_df = pd.DataFrame({'column_name': tennis_df.columns,
                             'percent_missing': percent_missing})
display(missing_value_df)

,column_name,percent_missing
ATP,ATP,0.0
Location,Location,0.0
Tournament,Tournament,0.0
Date,Date,0.0
Series,Series,0.0
Court,Court,0.0
Surface,Surface,0.0
Round,Round,0.0
Winner,Winner,0.0
Loser,Loser,0.0


In [15]:
len(tennis_df)

49775

In [16]:
tennis_df.reset_index(inplace=True, drop=True)

In [17]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7.0,133,2002,1.100,6.500
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6.0,56,2002,1.444,2.625
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41.0,20,2002,2.625,1.444
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21.0,150,2002,1.533,2.375
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13.0,107,2002,1.143,5.000


In [18]:
# convert ranks to ints
tennis_df['WRank'] = tennis_df['WRank'].astype(int)
tennis_df['LRank'] = tennis_df['LRank'].astype(int)

In [19]:
tennis_df.dtypes

ATP                    int64
Location              object
Tournament            object
Date          datetime64[ns]
Series                object
Court                 object
Surface               object
Round                 object
Winner                object
Loser                 object
WRank                  int32
LRank                  int32
Year                   int64
B365W                float64
B365L                float64
dtype: object

In [20]:
# this code identifies which tournaments don't have semifinals or finals

unique_tournaments = tennis_df['Tournament'].unique()
unique_rounds = tennis_df['Round'].unique()
display(unique_rounds)

def find_no_rounds_available(tournaments_list, round_list):

    for tournament in tournaments_list:

        tournament_df = tennis_df[tennis_df['Tournament']==tournament]


        if tournament == "Queens Club":
            display(tournament_df)

        for round_num in round_list:

            round_df = tournament_df[tournament_df['Round']==round_num]

            if len(round_df)==0:
                print(tournament, f"NO {round_num}")


find_no_rounds_available(
    tournaments_list=unique_tournaments, 
    round_list=['Semifinals', 'The Final'])

array(['1st Round', '2nd Round', '3rd Round', '4th Round',
       'Quarterfinals', 'Semifinals', 'The Final', 'Round Robin'],
      dtype=object)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
7,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Godwin N.,Gasquet R.,111,281,2002,1.533,2.375
8,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Lee H.T.,Larsson M.,117,142,2002,2.625,1.444
9,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Morrison J.,Ilie A.,98,176,2002,1.571,2.250
10,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Spadea V.,Llodra M.,73,103,2002,2.000,1.727


Queens Club NO Semifinals
Queens Club NO The Final
Zhuhai Open NO Semifinals


In [21]:
queens_club = tennis_df[tennis_df['Tournament']=='Queens Club']
display('Queens Club', queens_club)
display("Queens club", queens_club['Round'].unique())
zhuhai = tennis_df[tennis_df['Tournament']=='Zhuhai Open']
display('Zhuhai Open', zhuhai)
display("Zhuhai", zhuhai['Round'].unique())

'Queens Club'

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
7,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Godwin N.,Gasquet R.,111,281,2002,1.533,2.375
8,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Lee H.T.,Larsson M.,117,142,2002,2.625,1.444
9,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Morrison J.,Ilie A.,98,176,2002,1.571,2.250
10,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Spadea V.,Llodra M.,73,103,2002,2.000,1.727


'Queens club'

array(['1st Round'], dtype=object)

'Zhuhai Open'

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
45795,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Norrie C.,Gojowczyk P.,68,92,2019,1.53,2.37
45796,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Ramos-Vinolas A.,Nishioka Y.,46,60,2019,2.75,1.40
45797,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Wu D.,Ito T.,339,141,2019,2.37,1.53
45798,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Mannarino A.,Zhang Ze.,61,333,2019,1.28,3.50
45799,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Andujar P.,Johnson S.,53,94,2019,2.37,1.53
45800,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Kecmanovic M.,Ruud C.,50,59,2019,1.61,2.20
45801,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Zhang Zh.,Koepfer D.,224,85,2019,3.50,1.28
45802,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Dzumhur D.,Cecchinato M.,93,67,2019,1.40,2.75
45803,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,De Minaur A.,Millman J.,31,79,2019,1.44,2.62
45804,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Murray A.,Sandgren T.,413,69,2019,1.66,2.10


'Zhuhai'

array(['1st Round', '2nd Round', 'Quarterfinals', 'The Final'],
      dtype=object)

In [22]:
tennis_df[tennis_df['Tournament']=='Zhuhai Open'].head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L
45795,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Norrie C.,Gojowczyk P.,68,92,2019,1.53,2.37
45796,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Ramos-Vinolas A.,Nishioka Y.,46,60,2019,2.75,1.40
45797,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Wu D.,Ito T.,339,141,2019,2.37,1.53
45798,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Mannarino A.,Zhang Ze.,61,333,2019,1.28,3.50
45799,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Andujar P.,Johnson S.,53,94,2019,2.37,1.53


In [23]:
# dropping both of these tournaments from our set because they don't have both a semifinals and a finals
# we need both of these to measure how much pressure exists in a match

dropped_tournaments = ['Queens club', 'Zhuhai Open']

tennis_df = tennis_df[~tennis_df['Tournament'].isin(dropped_tournaments)]

In [24]:
assert(len(tennis_df[tennis_df['Tournament']=='Zhuhai Open'])==0)
assert(len(tennis_df[tennis_df['Tournament']=='Queens club'])==0)

In [25]:
# ranks final as highest pressure
# ranks semifinal as next highest
# categorizes everything else as not pressure

def tournament_rank_helper(round_tn):

    round_rank = None
    if round_tn == "The Final":
        round_rank = 1
    elif round_tn == "2nd Round":
        round_rank = 2
    else:
        round_rank = 3

    return round_rank


tennis_df['Tournament Rank'] = tennis_df['Round'].apply(tournament_rank_helper)


In [26]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L,Tournament Rank
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7,133,2002,1.100,6.500,3
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6,56,2002,1.444,2.625,3
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41,20,2002,2.625,1.444,3
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21,150,2002,1.533,2.375,3
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13,107,2002,1.143,5.000,3


In [27]:
display(tennis_df['Round'].unique())

array(['1st Round', '2nd Round', '3rd Round', '4th Round',
       'Quarterfinals', 'Semifinals', 'The Final', 'Round Robin'],
      dtype=object)

In [28]:
# convert European odds into probabilities
# B365 factors in the VIM (how much the book makes)

tennis_df['B365W_prob'] = (1/tennis_df['B365W']).round(3)
tennis_df['B365L_prob'] = (1/tennis_df['B365L']).round(3)
tennis_df['B365_total_prob'] = tennis_df['B365W_prob'] + tennis_df['B365L_prob']
tennis_df['B365_diff_from_1'] = tennis_df['B365_total_prob'] - 1

# This takes VIM out to just get the real probabilities
tennis_df['real_prob_W'] = tennis_df['B365W_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_prob_L'] = tennis_df['B365L_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_total_prob'] = tennis_df['real_prob_W'] + tennis_df['real_prob_L']
tennis_df['real_diff_from_1'] = tennis_df['real_total_prob'] - 1

In [29]:
tennis_df.head(5)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,Year,B365W,B365L,Tournament Rank,B365W_prob,B365L_prob,B365_total_prob,B365_diff_from_1,real_prob_W,real_prob_L,real_total_prob,real_diff_from_1
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7,133,2002,1.100,6.500,3,0.909,0.154,1.063,0.063,0.8775,0.1225,1.0,0.0
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6,56,2002,1.444,2.625,3,0.693,0.381,1.074,0.074,0.6560,0.3440,1.0,0.0
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41,20,2002,2.625,1.444,3,0.381,0.693,1.074,0.074,0.3440,0.6560,1.0,0.0
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21,150,2002,1.533,2.375,3,0.652,0.421,1.073,0.073,0.6155,0.3845,1.0,0.0
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13,107,2002,1.143,5.000,3,0.875,0.200,1.075,0.075,0.8375,0.1625,1.0,0.0


In [30]:
# isolate higher rank
# isolate loser rank
# locate player in df before time series
# find out percentage in win vs loser column
# find surface and other features

In [31]:
tennis_df_historical = tennis_df.copy()

In [32]:
drop_cols = ['ATP', 'Tournament', 'Round', 'Year']

tennis_df = tennis_df.drop(drop_cols, axis=1)

In [33]:
import datetime

In [34]:
# creates player dictionary with basic info
def player_dict_generator(name, rank, prob):
    
    player_dict = {}
    player_dict['name'] = name
    player_dict['rank'] = rank
    player_dict['prob'] = prob
    
    return player_dict

In [35]:
# queries into whole dataset to filter so we're only looking at matches of a player before the match of interest
def historical_match_data(player_name, date_of_match):
#     isolated all matches a player has played in (whole dataset)
    player_rank_history = tennis_df_historical.loc[(tennis_df_historical['Winner'] == player_name) | (tennis_df_historical['Loser'] == player_name)]
#     filter this so we're only looking at historical matches

    player_rank_history = player_rank_history.loc[player_rank_history['Date'] < date_of_match]

    return player_rank_history

In [36]:
# returns winning percentage for given df
# this function should only be called after historical_match_data()
def winning_percentage(player_df, player_name):
    
#     find where player wins, loses, then get percentage
    appears_winner = len(player_df.loc[player_df['Winner']==player_name])
    appears_loser = len(player_df.loc[player_df['Loser']==player_name])
    assert(appears_winner + appears_loser == len(player_df))
    winning_percentage = appears_winner/len(player_df)
    
    return winning_percentage

In [37]:
# returns multiple winning percentages: based on court type (outdoor, indoor), surface (grass, clay, etc), and series (Grand slam, etc.)
def winning_percentage_stats(player_history, player_name, court_type, surface_type, series_type, last_n_games):
    
#     isolate court type
    player_court_type = player_history.loc[player_history['Court']==court_type]
    
#     assert statements to check we isolated correctly
#     assert(len(set(player_court_type['Court']))==1)
#     assert(set(player_court_type['Court']).pop()==court_type)
    
#     repeat same as above for surface and series
    player_surface_type = player_history.loc[player_history['Surface']==surface_type]
    
#     assert(len(set(player_surface_type['Surface']))==1)
#     assert(set(player_surface_type['Surface']).pop()==surface_type)
    
    player_series_type = player_history.loc[player_history['Series']==series_type]
    
#     assert(len(set(player_series_type['Series']))==1)
#     assert(set(player_series_type['Series']).pop()==series_type)
    
    if (len(player_court_type) < last_n_games) or (len(player_surface_type) < last_n_games) or (len(player_series_type) < last_n_games):
        return (None, None, None)
    else:
    #     put the above dataframes into the winning_percentage() function to calculate winning percentage
    #     based on these factors
        court_type_winning_pct = winning_percentage(
            player_df=player_court_type, 
            player_name=player_name)

        surface_type_winning_pct = winning_percentage(
            player_df=player_surface_type, 
            player_name=player_name)

        series_type_winning_pct = winning_percentage(
            player_df=player_series_type, 
            player_name=player_name)

       # print(player_name, court_type, surface_type, series_type)
    #     display(player_court_type)
    #     display(player_surface_type)
    #     display(player_series_type)

    #     return results as a tuple
        return court_type_winning_pct, surface_type_winning_pct, series_type_winning_pct

In [38]:
# returns a bunch of stats for a player
def player_stats(player, last_n_games, last_n_days, match_date, court_type, surface_type, series_type):
    
#     isolate player name, rank, win prob
    name = player['name']
    rank = player['rank']
    prob = player['prob']
    
#     get player historical data, calculate days since last match
#     player_history = historical_match_data(player_name=name)
    player_history = historical_match_data(
        player_name=name,
        date_of_match=match_date
    )
    
    if len(player_history) < last_n_games:
        return None
        
    else:
    #     .date()
        # player_last_match = player_history.iloc[-1]['Date']
        # #print(match_date, player_last_match)
        # player_days_since_last_match = (match_date - player_last_match).days


        last_n_days_dt = datetime.timedelta(last_n_days)
        n_days_before_match = match_date - last_n_days_dt

        # player_historical_match_days = historical_match_data(
        #     player_name=name, 
        #     date_of_match=match_date, 
        #     start_date=n_days_before_match
        # )

        player_historical_match_days = player_history.loc[
            (player_history['Date'] < match_date) & 
            ((player_history['Date'] > n_days_before_match))
        ]


        amount_matches_n_days_before = len(player_historical_match_days)
        match_per_day_pecentage = amount_matches_n_days_before/last_n_days

        # print(match_date, last_n_days, last_n_days_dt, n_days_before_match)
        # display(player_historical_match_days, match_per_day_pecentage)

        

    #     find player last n games (to calculate win percentage for recent matches)
        player_last_n_games = player_history[-last_n_games:]

    #     get player win pct
        player_win_pct = winning_percentage(
            player_df=player_last_n_games,
            player_name=name)

    #     get player win percentage on the court the match is on, the sruface type of the match, and the series type
        player_court_type_win_pct, player_surface_type_win_pct, player_series_type_win_pct = winning_percentage_stats(
            player_history=player_history, 
            player_name=name, 
            court_type=court_type,
            surface_type=surface_type,
            series_type=series_type,
            last_n_games=last_n_games,
        )
        
        if (player_court_type_win_pct, player_surface_type_win_pct, player_series_type_win_pct) == (None, None, None):
            return None
        else:
    #     return all this info for the player
            ret_dict = {
                'name':name,
                'rank':rank,
                'prob':prob,
                'matches_per_day_rate': match_per_day_pecentage,
                'win_pct': player_win_pct,
                'court_type_win_pct':player_court_type_win_pct,
                'surface_type_win_pct':player_surface_type_win_pct,
                'series_type_win_pct':player_series_type_win_pct
            }

            return ret_dict

In [39]:
from datetime import date

# this creates the dataset we can use for ML
# with feature engineering

def ML_df_function(df, start_row=None, end_row=None):

#     set up return DF with our new formatted data
#     to find what columns to put for final_df:
#     display(returned_row.keys())
    final_columns = ['Location', 'Date', 'Series', 'Court', 'Surface', 'HR_win?', 'HR_name', 'LR_name', 
    'higher_ranked_rank', 'higher_ranked_prob', 'higher_ranked_matches_per_day_rate', 'higher_ranked_win_pct', 
    'higher_ranked_court_type_win_pct', 'higher_ranked_surface_type_win_pct', 'higher_ranked_series_type_win_pct', 
    'lower_ranked_rank', 'lower_ranked_prob', 'lower_ranked_matches_per_day_rate', 'lower_ranked_win_pct', 
    'lower_ranked_court_type_win_pct', 'lower_ranked_surface_type_win_pct', 
    'lower_ranked_series_type_win_pct', 'diff_rank', 'diff_prob', 'diff_matches_per_day_rate', 'diff_win_pct', 
    'diff_court_type_win_pct', 'diff_surface_type_win_pct', 'diff_series_type_win_pct']
    final_df = pd.DataFrame(columns = final_columns)


    num_rows = end_row-start_row
    print(f"{num_rows} total rows in dataset")
    current_row = 0

#     tells us progress of processing
    for i in range(start_row,end_row):

        current_row += 1
        if (current_row % 5000==0):
            print(f"{current_row} rows processed.")

#         create dictionary to return the new row
        returned_row = {}
        returned_row['Location'] = df.loc[i,'Location']
        returned_row['Date'] = df.loc[i,'Date']
        returned_row['Series'] = df.loc[i,'Series']
        returned_row['Court'] = df.loc[i,'Court']
        returned_row['Surface'] = df.loc[i,'Surface']


#         display original df row
#         display(row.to_frame().T)

#         get date of the match
        date_of_match = df.loc[i,'Date']

#         set up information about the winner and loser
        winner = player_dict_generator(
            name=df.loc[i,'Winner'], 
            rank=df.loc[i,'WRank'], 
            prob=df.loc[i,'real_prob_W'])


        loser = player_dict_generator(
            name=df.loc[i,'Loser'], 
            rank=df.loc[i,'LRank'], 
            prob=df.loc[i,'real_prob_L'])


#         figure out the higher and lower rank
        higher_rank = {}
        lower_rank = {}
        if winner['rank'] < loser['rank']:
            higher_rank = winner
            lower_rank = loser
        else:
            higher_rank = loser
            lower_rank = winner



#         get player stats for higher and lower rank
        higher_ranked_player_stats = player_stats(
            player=higher_rank, 
            last_n_games=10,
            last_n_days=60,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series']

        )

        lower_ranked_player_stats = player_stats(
            player=lower_rank, 
            last_n_games=10,
            last_n_days=60,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series']
        )
        
        if (higher_ranked_player_stats == None) or (lower_ranked_player_stats == None):
            i += 1
        else:
    #         display(higher_ranked_player_stats)
    #         display(lower_ranked_player_stats)

    #         for each stat, append this to the returned fr (row to be 
    #         added to new ML df)


            hr_name = higher_ranked_player_stats['name']
    #         print(hr_name)
            winner = df.loc[i,'Winner']
    #         print(winner)

            if hr_name == winner:
                returned_row['HR_win?']=1
            else:
                returned_row['HR_win?']=0

    #         print(hr_name==winner)
    #         deletes name so we are only taking diff on numeric columns

            returned_row['HR_name'] = higher_ranked_player_stats['name']
            returned_row['LR_name'] = lower_ranked_player_stats['name']

            del higher_ranked_player_stats['name']
            del lower_ranked_player_stats['name']


            value_keys = list(higher_ranked_player_stats.keys())
    #         display(value_keys)
    #         print("VALUES")
            high_rank_list = list(higher_ranked_player_stats.values())
            low_rank_list = list(lower_ranked_player_stats.values())

            for key, stat in zip(value_keys, high_rank_list):
                returned_row[f'higher_ranked_{key}'] = stat

            for key, stat in zip(value_keys, low_rank_list):
                returned_row[f'lower_ranked_{key}'] = stat

    #         display(high_rank_list)
    #         display(low_rank_list)
            diffs = [a_i - b_i for a_i, b_i in zip(high_rank_list, low_rank_list)]
    #         print("diffs")
    #         display(diffs)


            for key, diff in zip(value_keys, diffs):
                returned_row[f'diff_{key}'] = diff


            # display(returned_row.keys())
            final_df = final_df.append(returned_row, ignore_index=True)
    return final_df


In [40]:
tennis_df = tennis_df.reset_index(drop=True)

In [41]:
tennis_df.tail()

,Location,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,B365W,B365L,Tournament Rank,B365W_prob,B365L_prob,B365_total_prob,B365_diff_from_1,real_prob_W,real_prob_L,real_total_prob,real_diff_from_1
49745,Vienna,2021-10-29,ATP500,Indoor,Hard,Tiafoe F.,Schwartzman D.,49,16,2.20,1.66,3,0.455,0.602,1.057,0.057,0.4265,0.5735,1.0,0.0
49746,Vienna,2021-10-29,ATP500,Indoor,Hard,Sinner J.,Ruud C.,11,8,1.40,3.00,3,0.714,0.333,1.047,0.047,0.6905,0.3095,1.0,0.0
49747,Vienna,2021-10-30,ATP500,Indoor,Hard,Zverev A.,Alcaraz C.,4,42,1.28,3.75,3,0.781,0.267,1.048,0.048,0.7570,0.2430,1.0,0.0
49748,Vienna,2021-10-30,ATP500,Indoor,Hard,Tiafoe F.,Sinner J.,49,11,4.00,1.25,3,0.250,0.800,1.050,0.050,0.2250,0.7750,1.0,0.0
49749,Vienna,2021-10-31,ATP500,Indoor,Hard,Zverev A.,Tiafoe F.,4,49,1.20,4.50,1,0.833,0.222,1.055,0.055,0.8055,0.1945,1.0,0.0


In [42]:


ML_dataframe = ML_df_function(
    df=tennis_df, 
    start_row=0, end_row=len(tennis_df)
    # start_row=5000, end_row=5005
)


# Features:

# who won, along with all the drf features
# HRwin? 0 1

#     last 10 winning
#     number days sine last match
#     surface type
#     outdoor vs indoor
#     winning percentage in type of series
#     winning percentage in round
#     How much rank is chancged before match

49750 total rows in dataset
5000 rows processed.
10000 rows processed.
15000 rows processed.
20000 rows processed.
25000 rows processed.
30000 rows processed.
35000 rows processed.
40000 rows processed.
45000 rows processed.


In [43]:
tennis_df.tail()

,Location,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,B365W,B365L,Tournament Rank,B365W_prob,B365L_prob,B365_total_prob,B365_diff_from_1,real_prob_W,real_prob_L,real_total_prob,real_diff_from_1
49745,Vienna,2021-10-29,ATP500,Indoor,Hard,Tiafoe F.,Schwartzman D.,49,16,2.20,1.66,3,0.455,0.602,1.057,0.057,0.4265,0.5735,1.0,0.0
49746,Vienna,2021-10-29,ATP500,Indoor,Hard,Sinner J.,Ruud C.,11,8,1.40,3.00,3,0.714,0.333,1.047,0.047,0.6905,0.3095,1.0,0.0
49747,Vienna,2021-10-30,ATP500,Indoor,Hard,Zverev A.,Alcaraz C.,4,42,1.28,3.75,3,0.781,0.267,1.048,0.048,0.7570,0.2430,1.0,0.0
49748,Vienna,2021-10-30,ATP500,Indoor,Hard,Tiafoe F.,Sinner J.,49,11,4.00,1.25,3,0.250,0.800,1.050,0.050,0.2250,0.7750,1.0,0.0
49749,Vienna,2021-10-31,ATP500,Indoor,Hard,Zverev A.,Tiafoe F.,4,49,1.20,4.50,1,0.833,0.222,1.055,0.055,0.8055,0.1945,1.0,0.0


In [44]:
ML_dataframe.tail()

,Location,Date,Series,Court,Surface,HR_win?,HR_name,LR_name,higher_ranked_rank,higher_ranked_prob,higher_ranked_matches_per_day_rate,higher_ranked_win_pct,higher_ranked_court_type_win_pct,higher_ranked_surface_type_win_pct,higher_ranked_series_type_win_pct,lower_ranked_rank,lower_ranked_prob,lower_ranked_matches_per_day_rate,lower_ranked_win_pct,lower_ranked_court_type_win_pct,lower_ranked_surface_type_win_pct,lower_ranked_series_type_win_pct,diff_rank,diff_prob,diff_matches_per_day_rate,diff_win_pct,diff_court_type_win_pct,diff_surface_type_win_pct,diff_series_type_win_pct
26383,Vienna,2021-10-29,ATP500,Indoor,Hard,1,Zverev A.,Auger-Aliassime F.,4,0.7750,0.200000,0.8,0.643836,0.685106,0.676190,12,0.2250,0.133333,0.7,0.580645,0.591837,0.619048,-8,0.550,0.066667,0.1,0.063190,0.093270,0.057143
26384,Vienna,2021-10-29,ATP500,Indoor,Hard,0,Schwartzman D.,Tiafoe F.,16,0.5735,0.266667,0.8,0.607843,0.573964,0.563636,49,0.4265,0.166667,0.7,0.466667,0.479167,0.406250,-33,0.147,0.100000,0.1,0.141176,0.094798,0.157386
26385,Vienna,2021-10-29,ATP500,Indoor,Hard,0,Ruud C.,Sinner J.,8,0.3095,0.166667,0.8,0.578947,0.471698,0.541667,11,0.6905,0.283333,0.9,0.736842,0.702703,0.727273,-3,-0.381,-0.116667,-0.1,-0.157895,-0.231005,-0.185606
26386,Vienna,2021-10-30,ATP500,Indoor,Hard,0,Sinner J.,Tiafoe F.,11,0.7750,0.283333,0.9,0.743590,0.706667,0.739130,49,0.2250,0.166667,0.7,0.483871,0.482759,0.424242,-38,0.550,0.116667,0.2,0.259719,0.223908,0.314888
26387,Vienna,2021-10-31,ATP500,Indoor,Hard,1,Zverev A.,Tiafoe F.,4,0.8055,0.216667,0.8,0.653333,0.687764,0.682243,49,0.1945,0.166667,0.7,0.500000,0.486301,0.441176,-45,0.611,0.050000,0.1,0.153333,0.201462,0.241067


In [46]:
HR_win_col = ML_dataframe.pop('HR_win?')
ML_dataframe.insert(len(ML_dataframe.columns), 'HR_win?', HR_win_col)

In [47]:
ML_dataframe.head()

,Location,Date,Series,Court,Surface,HR_name,LR_name,higher_ranked_rank,higher_ranked_prob,higher_ranked_matches_per_day_rate,higher_ranked_win_pct,higher_ranked_court_type_win_pct,higher_ranked_surface_type_win_pct,higher_ranked_series_type_win_pct,lower_ranked_rank,lower_ranked_prob,lower_ranked_matches_per_day_rate,lower_ranked_win_pct,lower_ranked_court_type_win_pct,lower_ranked_surface_type_win_pct,lower_ranked_series_type_win_pct,diff_rank,diff_prob,diff_matches_per_day_rate,diff_win_pct,diff_court_type_win_pct,diff_surface_type_win_pct,diff_series_type_win_pct,HR_win?
0,Chennai,2003-01-04,International,Outdoor,Hard,Chela J.I.,Kucera K.,23,0.5965,0.050000,0.6,0.769231,0.684211,0.823529,83,0.4035,0.050000,0.7,0.500000,0.666667,0.600000,-60,0.193,0.000000,-0.1,0.269231,0.017544,0.223529,0
1,Chennai,2003-01-05,International,Outdoor,Hard,Srichaphan P.,Kucera K.,16,0.7210,0.066667,0.9,0.733333,0.777778,0.857143,83,0.2790,0.066667,0.8,0.533333,0.687500,0.636364,-67,0.442,0.000000,0.1,0.200000,0.090278,0.220779,1
2,Doha,2003-01-02,International,Outdoor,Hard,Federer R.,Gambill J.M.,6,0.7460,0.100000,0.7,0.500000,0.714286,0.600000,42,0.2540,0.033333,0.4,0.650000,0.636364,0.642857,-36,0.492,0.066667,0.3,-0.150000,0.077922,-0.042857,0
3,Doha,2003-01-02,International,Outdoor,Hard,Youzhny M.,Vicente F.,32,0.6750,0.033333,0.8,0.764706,0.750000,0.750000,58,0.3250,0.016667,0.3,0.411765,0.500000,0.300000,-26,0.350,0.016667,0.5,0.352941,0.250000,0.450000,1
4,Doha,2003-01-03,International,Outdoor,Hard,Youzhny M.,Gambill J.M.,32,0.5000,0.050000,0.9,0.777778,0.769231,0.761905,42,0.5000,0.050000,0.5,0.666667,0.652174,0.666667,-10,0.000,0.000000,0.4,0.111111,0.117057,0.095238,0


In [48]:
display(ML_dataframe)

,Location,Date,Series,Court,Surface,HR_name,LR_name,higher_ranked_rank,higher_ranked_prob,higher_ranked_matches_per_day_rate,higher_ranked_win_pct,higher_ranked_court_type_win_pct,higher_ranked_surface_type_win_pct,higher_ranked_series_type_win_pct,lower_ranked_rank,lower_ranked_prob,lower_ranked_matches_per_day_rate,lower_ranked_win_pct,lower_ranked_court_type_win_pct,lower_ranked_surface_type_win_pct,lower_ranked_series_type_win_pct,diff_rank,diff_prob,diff_matches_per_day_rate,diff_win_pct,diff_court_type_win_pct,diff_surface_type_win_pct,diff_series_type_win_pct,HR_win?
0,Chennai,2003-01-04,International,Outdoor,Hard,Chela J.I.,Kucera K.,23,0.5965,0.050000,0.6,0.769231,0.684211,0.823529,83,0.4035,0.050000,0.7,0.500000,0.666667,0.600000,-60,0.193,0.000000,-0.1,0.269231,0.017544,0.223529,0
1,Chennai,2003-01-05,International,Outdoor,Hard,Srichaphan P.,Kucera K.,16,0.7210,0.066667,0.9,0.733333,0.777778,0.857143,83,0.2790,0.066667,0.8,0.533333,0.687500,0.636364,-67,0.442,0.000000,0.1,0.200000,0.090278,0.220779,1
2,Doha,2003-01-02,International,Outdoor,Hard,Federer R.,Gambill J.M.,6,0.7460,0.100000,0.7,0.500000,0.714286,0.600000,42,0.2540,0.033333,0.4,0.650000,0.636364,0.642857,-36,0.492,0.066667,0.3,-0.150000,0.077922,-0.042857,0
3,Doha,2003-01-02,International,Outdoor,Hard,Youzhny M.,Vicente F.,32,0.6750,0.033333,0.8,0.764706,0.750000,0.750000,58,0.3250,0.016667,0.3,0.411765,0.500000,0.300000,-26,0.350,0.016667,0.5,0.352941,0.250000,0.450000,1
4,Doha,2003-01-03,International,Outdoor,Hard,Youzhny M.,Gambill J.M.,32,0.5000,0.050000,0.9,0.777778,0.769231,0.761905,42,0.5000,0.050000,0.5,0.666667,0.652174,0.666667,-10,0.000,0.000000,0.4,0.111111,0.117057,0.095238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26383,Vienna,2021-10-29,ATP500,Indoor,Hard,Zverev A.,Auger-Aliassime F.,4,0.7750,0.200000,0.8,0.643836,0.685106,0.676190,12,0.2250,0.133333,0.7,0.580645,0.591837,0.619048,-8,0.550,0.066667,0.1,0.063190,0.093270,0.057143,1
26384,Vienna,2021-10-29,ATP500,Indoor,Hard,Schwartzman D.,Tiafoe F.,16,0.5735,0.266667,0.8,0.607843,0.573964,0.563636,49,0.4265,0.166667,0.7,0.466667,0.479167,0.406250,-33,0.147,0.100000,0.1,0.141176,0.094798,0.157386,0
26385,Vienna,2021-10-29,ATP500,Indoor,Hard,Ruud C.,Sinner J.,8,0.3095,0.166667,0.8,0.578947,0.471698,0.541667,11,0.6905,0.283333,0.9,0.736842,0.702703,0.727273,-3,-0.381,-0.116667,-0.1,-0.157895,-0.231005,-0.185606,0
26386,Vienna,2021-10-30,ATP500,Indoor,Hard,Sinner J.,Tiafoe F.,11,0.7750,0.283333,0.9,0.743590,0.706667,0.739130,49,0.2250,0.166667,0.7,0.483871,0.482759,0.424242,-38,0.550,0.116667,0.2,0.259719,0.223908,0.314888,0


In [49]:
import numpy as np
np.percentile(tennis_df["LRank"], 90)

168.0

In [50]:
ML_dataframe.to_csv("complete_data.csv")

In [ ]:
# can either just focus on higher rank, and get info about his past and ooppondent's past
# or can take the difference in odds, which require us breaking each player into 2 rows, creating a regression on each, then subtracting the odds difference

# think the higher rank version is better option, focuses on just 1 regression and 1 decision

In [ ]:
# tennis_df.to_excel("tennis_data_2001_2021.xlsx") 